In [1]:
!pip install cryptography
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.backends import default_backend
import base64
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding, rsa
from cryptography.exceptions import InvalidSignature
from pathlib import Path
import os
from cryptography.hazmat.backends import default_backend
import requests
import datetime

In [7]:
# Cell 1 — load KALSHI keys from .env in the same directory as this notebook


# --- Try to use python-dotenv if it's installed (recommended) ---
try:
    from dotenv import load_dotenv  # pip install python-dotenv
    env_path = Path.cwd() / ".env"   # assumes notebook is run from the same directory as .env
    if not env_path.exists():
        raise FileNotFoundError(f".env not found at: {env_path}")
    load_dotenv(dotenv_path=env_path, override=False)
except ImportError:
    # --- Fallback: minimal .env parser (no extra install needed) ---
    env_path = Path.cwd() / ".env"
    if not env_path.exists():
        raise FileNotFoundError(f".env not found at: {env_path}")

    for line in env_path.read_text().splitlines():
        line = line.strip()
        if not line or line.startswith("#") or "=" not in line:
            continue
        k, v = line.split("=", 1)
        k = k.strip()
        v = v.strip().strip('"').strip("'")
        os.environ.setdefault(k, v)  # don't overwrite if already set

# --- Read your keys into variables ---
public_key = os.getenv("KALSHI-ACCESS-KEY-DEMO")

missing = [k for k, v in {
    "KALSHI-ACCESS-KEY": public_key
}.items() if not v]

if missing:
    raise KeyError(f"Missing env var(s): {missing}. Check your .env formatting and key names.")

print("Loaded Kalshi keys ✅")  # intentionally not printing the secret values


def load_private_key_from_file(file_path):
    with open(file_path, "rb") as key_file:
        private_key = serialization.load_pem_private_key(
            key_file.read(),
            password=None,  # or provide a password if your key is encrypted
            backend=default_backend()
        )
    return private_key


def sign_pss_text(private_key: rsa.RSAPrivateKey, text: str) -> str:
    message = text.encode('utf-8')
    try:
        signature = private_key.sign(
            message,
            padding.PSS(
                mgf=padding.MGF1(hashes.SHA256()),
                salt_length=padding.PSS.DIGEST_LENGTH
            ),
            hashes.SHA256()
        )
        return base64.b64encode(signature).decode('utf-8')
    except InvalidSignature as e:
        raise ValueError("RSA sign PSS failed") from e
    

def make_authenticated_request(path, method='GET', params=None):
    """Make an authenticated request to Kalshi demo API."""
    current_time = datetime.datetime.now()
    timestamp = current_time.timestamp()
    current_time_milliseconds = int(timestamp * 1000)
    timestamp_str = str(current_time_milliseconds)
    
    private_key = load_private_key_from_file('pbhaskarademo.txt')
    
    base_url = 'https://demo-api.kalshi.co'
    
    # Strip query parameters from path before signing
    path_without_query = path.split('?')[0]
    msg_string = timestamp_str + method + path_without_query
    sig = sign_pss_text(private_key, msg_string)
    
    headers = {
        'KALSHI-ACCESS-KEY': public_key,
        'KALSHI-ACCESS-SIGNATURE': sig,
        'KALSHI-ACCESS-TIMESTAMP': timestamp_str
    }
    
    if method == 'GET':
        response = requests.get(base_url + path, headers=headers, params=params)
    else:
        response = requests.post(base_url + path, headers=headers, json=params)
    
    return response

Loaded Kalshi keys ✅


In [16]:
# ============================================================================
# FETCH NYC MAYOR PARTY WINNER MARKET - KXMAYORNYCPARTY-25
# ============================================================================

def fetch_market_summary_details(ticker):
    import time
    print("="*80)
    print("FETCHING MARKET")
    print("="*80)

    event_ticker = ticker #"KXMAYORNYCPARTY-25" #write event ticker here
    base_url = "https://api.elections.kalshi.com/trade-api/v2"

    # Extract series ticker (remove year suffix)
    series_ticker = event_ticker.rsplit('-', 1)[0]  # "KXMAYORNYCPARTY"

    try:
        # Fetch event and markets
        response = requests.get(f"{base_url}/events/{event_ticker}")
        
        if response.status_code == 200:
            event_data = response.json()
            event_info = event_data.get('event', {})
            markets = event_data.get('markets', [])
            
            print(f"\nEvent: {event_info.get('title', 'N/A')}")
            print(f"Category: {event_info.get('category', 'N/A')}")
            print(f"Series: {series_ticker}")
            print(f"Markets: {len(markets)}\n")
            
            if len(markets) == 0:
                print("❌ No markets found in this event")
            else:
                # Get timing info from first market (all should have same timing)
                first_market = markets[0]
                open_time = datetime.datetime.fromisoformat(first_market['open_time'].replace('Z', '+00:00'))
                close_time = datetime.datetime.fromisoformat(first_market['close_time'].replace('Z', '+00:00'))
                duration = close_time - open_time
                
                # Calculate 90% elapsed time
                time_90pct_elapsed = open_time + (duration * 0.90)
                time_remaining = close_time - time_90pct_elapsed
                
                print(f"📅 MARKET TIMELINE:")
                print(f"  Opened:            {open_time.strftime('%Y-%m-%d %H:%M:%S UTC')}")
                print(f"  90% elapsed at:    {time_90pct_elapsed.strftime('%Y-%m-%d %H:%M:%S UTC')}")
                print(f"  Closed:            {close_time.strftime('%Y-%m-%d %H:%M:%S UTC')}")
                print(f"  Duration:          {duration.total_seconds() / 3600 / 24:.2f} days")
                print(f"  Time at 90%:       {time_remaining.total_seconds() / 3600:.2f} hours remaining")
                
                target_ts = int(time_90pct_elapsed.timestamp())
                start_ts = int(open_time.timestamp())
                end_ts = int(close_time.timestamp())
                
                print(f"\n{'='*80}")
                print("PRICES AT 90% TIME ELAPSED FOR EACH MARKET")
                print("="*80)
                
                # Fetch prices for each market
                for market in markets:
                    ticker = market['ticker']
                    subtitle = market.get('subtitle', market.get('title', 'N/A'))
                    result = market.get('result', 'N/A')
                    volume = float(market.get('volume', '0'))
                    
                    print(f"\n{subtitle}")
                    print(f"  Ticker: {ticker}")
                    print(f"  Result: {result.upper()}")
                    print(f"  Total Volume: ${volume:,.2f}")
                    
                    # Try to fetch historical candlestick data
                    candlestick_data = None
                    for period in [60, 1440]:  # Try 1-hour and daily
                        try:
                            candle_url = f"{base_url}/series/{series_ticker}/markets/{ticker}/candlesticks"
                            candle_response = requests.get(
                                candle_url,
                                params={
                                    "start_ts": start_ts,
                                    "end_ts": end_ts,
                                    "period_interval": period
                                }
                            )
                            if candle_response.status_code == 200:
                                candlestick_data = candle_response.json()
                                break
                        except:
                            continue
                    
                    if candlestick_data and 'candlesticks' in candlestick_data:
                        candles = candlestick_data['candlesticks']
                        
                        # Find candlestick closest to 90% elapsed
                        closest_candle = None
                        min_diff = float('inf')
                        
                        for candle in candles:
                            candle_ts = candle.get('end_period_ts', 0)
                            diff = abs(candle_ts - target_ts)
                            if diff < min_diff:
                                min_diff = diff
                                closest_candle = candle
                        
                        if closest_candle:
                            candle_time = datetime.datetime.fromtimestamp(
                                closest_candle['end_period_ts'], 
                                tz=datetime.timezone.utc
                            )
                            
                            price_obj = closest_candle.get('price', {})
                            yes_bid_obj = closest_candle.get('yes_bid', {})
                            yes_ask_obj = closest_candle.get('yes_ask', {})
                            
                            print(f"\n  📊 Prices at 90% (closest data: {candle_time.strftime('%Y-%m-%d %H:%M UTC')}):")
                            print(f"    Trade Price: {price_obj.get('close', 'N/A')}¢")
                            print(f"    YES Bid:     {yes_bid_obj.get('close', 'N/A')}¢")
                            print(f"    YES Ask:     {yes_ask_obj.get('close', 'N/A')}¢")
                        else:
                            print(f"  ⚠️  No candlestick data found near 90% time")
                    else:
                        print(f"  ⚠️  Candlestick data not available")
                        print(f"  Final state - YES: {market.get('yes_bid', 'N/A')}¢/{market.get('yes_ask', 'N/A')}¢  NO: {market.get('no_bid', 'N/A')}¢/{market.get('no_ask', 'N/A')}¢")
                    
                    time.sleep(0.2)  # Rate limiting
                
        else:
            print(f"\n❌ Error: HTTP {response.status_code}")
            print(f"Response: {response.text[:300]}")

    except Exception as e:
        print(f"\n❌ Error: {e}")
        import traceback
        traceback.print_exc()

    print("\n" + "="*80)

In [30]:
# ============================================================================
# GENERAL FUNCTION: FETCH CANDLESTICK DATA FOR ANY MARKET
# ============================================================================

def fetch_market_candlestick_data(event_ticker, period_intervals=[60, 1440], verbose=True):
    """
    Fetch all candlestick data for a given Kalshi event.
    
    Parameters:
    -----------
    event_ticker : str
        The event ticker (e.g., "KXMAYORNYCPARTY-25")
    period_intervals : list of int
        Candlestick periods to try in minutes (default: [60, 1440] for hourly and daily)
        Valid values: 1, 60, 1440
    verbose : bool
        Print progress information (default: True)
    
    Returns:
    --------
    pandas.DataFrame
        DataFrame with candlestick data for all markets in the event
        Columns: ticker, series_ticker, party/subtitle, timestamp, prices, volume, open_interest
    """
    import pandas as pd
    
    if verbose:
        print("="*80)
        print(f"FETCHING CANDLESTICK DATA FOR EVENT: {event_ticker}")
        print("="*80)
    
    base_url = "https://api.elections.kalshi.com/trade-api/v2"
    
    # Extract series ticker (remove year suffix)
    series_ticker = event_ticker.rsplit('-', 1)[0]
    
    all_candlestick_data = []
    
    try:
        # Fetch event and markets
        response = requests.get(f"{base_url}/events/{event_ticker}")
        
        if response.status_code == 200:
            event_data = response.json()
            markets = event_data.get('markets', [])
            
            if verbose:
                print(f"\nSeries: {series_ticker}")
                print(f"Found {len(markets)} markets in event")
                print("Fetching candlestick data for each market...\n")
            
            for i, market in enumerate(markets, 1):
                ticker = market['ticker']
                subtitle = market.get('subtitle', market.get('title', 'N/A'))
                
                # Get market timing
                open_time = datetime.datetime.fromisoformat(market['open_time'].replace('Z', '+00:00'))
                close_time = datetime.datetime.fromisoformat(market['close_time'].replace('Z', '+00:00'))
                start_ts = int(open_time.timestamp())
                end_ts = int(close_time.timestamp())
                
                if verbose:
                    print(f"[{i}/{len(markets)}] {subtitle} ({ticker})")
                
                # Try to fetch candlestick data with different periods
                for period in period_intervals:
                    try:
                        candle_url = f"{base_url}/series/{series_ticker}/markets/{ticker}/candlesticks"
                        candle_response = requests.get(
                            candle_url,
                            params={
                                "start_ts": start_ts,
                                "end_ts": end_ts,
                                "period_interval": period
                            }
                        )
                        
                        if candle_response.status_code == 200:
                            candlestick_json = candle_response.json()
                            candles = candlestick_json.get('candlesticks', [])
                            
                            if candles:
                                if verbose:
                                    print(f"  ✓ Got {len(candles)} candlesticks (period={period} min)")
                                
                                # Add to collection
                                for candle in candles:
                                    candle_time = datetime.datetime.fromtimestamp(
                                        candle['end_period_ts'], 
                                        tz=datetime.timezone.utc
                                    )
                                    
                                    # Extract price data
                                    price_obj = candle.get('price', {})
                                    yes_bid_obj = candle.get('yes_bid', {})
                                    yes_ask_obj = candle.get('yes_ask', {})
                                    
                                    all_candlestick_data.append({
                                        'ticker': ticker,
                                        'series_ticker': series_ticker,
                                        'subtitle': subtitle,
                                        'timestamp': candle_time,
                                        'timestamp_unix': candle['end_period_ts'],
                                        'period_minutes': period,
                                        # Trade prices (actual executed trades)
                                        'price_open': price_obj.get('open'),
                                        'price_high': price_obj.get('high'),
                                        'price_low': price_obj.get('low'),
                                        'price_close': price_obj.get('close'),
                                        # YES bid prices (buyer's side)
                                        'yes_bid_open': yes_bid_obj.get('open'),
                                        'yes_bid_high': yes_bid_obj.get('high'),
                                        'yes_bid_low': yes_bid_obj.get('low'),
                                        'yes_bid_close': yes_bid_obj.get('close'),
                                        # YES ask prices (seller's side)
                                        'yes_ask_open': yes_ask_obj.get('open'),
                                        'yes_ask_high': yes_ask_obj.get('high'),
                                        'yes_ask_low': yes_ask_obj.get('low'),
                                        'yes_ask_close': yes_ask_obj.get('close'),
                                        'volume': candle.get('volume', 0),
                                        'open_interest': candle.get('open_interest', 0)
                                    })
                                break
                            else:
                                if verbose:
                                    print(f"  ⚠️  No data for period={period}")
                        else:
                            if verbose:
                                print(f"  ⚠️  HTTP {candle_response.status_code} for period={period}")
                                if candle_response.status_code != 404:
                                    print(f"      Response: {candle_response.text[:200]}")
                            
                    except Exception as e:
                        if verbose:
                            print(f"  ✗ Error fetching period={period}: {e}")
                        continue
                
                time.sleep(0.2)  # Rate limiting
            
            # Create DataFrame
            if all_candlestick_data:
                df_candlesticks = pd.DataFrame(all_candlestick_data)
                
                if verbose:
                    print(f"\n{'='*80}")
                    print(f"✓ FETCHED {len(df_candlesticks)} TOTAL CANDLESTICKS")
                    print("="*80)
                    
                    print(f"\nDataFrame shape: {df_candlesticks.shape}")
                    print(f"Columns: {list(df_candlesticks.columns)}")
                    print(f"\nMarkets: {df_candlesticks['subtitle'].unique().tolist()}")
                    print(f"Date range: {df_candlesticks['timestamp'].min()} to {df_candlesticks['timestamp'].max()}")
                    
                    print(f"\n{'='*80}")
                    print("SAMPLE DATA (First 10 rows)")
                    print("="*80)
                    print(df_candlesticks.head(10).to_string())
                    print("\n" + "="*80)
                
                return df_candlesticks
            else:
                if verbose:
                    print("\n❌ No candlestick data retrieved")
                    print("="*80)
                return pd.DataFrame()
                
        else:
            if verbose:
                print(f"\n❌ Error fetching event: HTTP {response.status_code}")
                print("="*80)
            return pd.DataFrame()

    except Exception as e:
        if verbose:
            print(f"\n❌ Error: {e}")
            import traceback
            traceback.print_exc()
            print("="*80)
        return pd.DataFrame()


# Example usage:
df = fetch_market_candlestick_data("KXMAYORNYCPARTY-25", [1])
# df = fetch_market_candlestick_data("KXFEDDECISION-25SEP", period_intervals=[1440], verbose=False)

FETCHING CANDLESTICK DATA FOR EVENT: KXMAYORNYCPARTY-25

Series: KXMAYORNYCPARTY
Found 5 markets in event
Fetching candlestick data for each market...

[1/5] :: Democratic Nominee (KXMAYORNYCPARTY-25-D)
  ⚠️  HTTP 400 for period=1
      Response: {"error":{"code":"bad_request","message":"bad request","details":"requested time range with candlesticks: 521778.966667, max candlesticks: 5000"}}
[2/5] :: Republican Nominee (KXMAYORNYCPARTY-25-R)
  ⚠️  HTTP 400 for period=1
      Response: {"error":{"code":"bad_request","message":"bad request","details":"requested time range with candlesticks: 521778.966667, max candlesticks: 5000"}}
[3/5] :: Independent (KXMAYORNYCPARTY-25-EADA)
  ⚠️  HTTP 400 for period=1
      Response: {"error":{"code":"bad_request","message":"bad request","details":"requested time range with candlesticks: 197418.966667, max candlesticks: 5000"}}
[4/5] :: Independent (KXMAYORNYCPARTY-25-JWAL)
  ⚠️  HTTP 400 for period=1
      Response: {"error":{"code":"bad_request","mes

In [ ]:
# ============================================================================
# EXAMPLE: FETCH NYC MAYOR CANDLESTICK DATA
# ============================================================================

# Fetch candlestick data for NYC Mayor race
df_nyc_mayor = fetch_market_candlestick_data("KXMAYORNYCPARTY-25")

# Data is now in df_nyc_mayor DataFrame

In [31]:
# ============================================================================
# FETCH AND FORWARD-FILL 1-MINUTE CANDLESTICKS - NYC MAYOR
# ============================================================================

import pandas as pd
import numpy as np
from datetime import datetime, timezone

print("="*80)
print("FETCHING 1-MINUTE CANDLESTICKS WITH FORWARD FILL")
print("="*80)

event_ticker = "KXMAYORNYCPARTY-25"
base_url = "https://api.elections.kalshi.com/trade-api/v2"
series_ticker = event_ticker.rsplit('-', 1)[0]

try:
    # Get all markets in the event
    response = requests.get(f"{base_url}/events/{event_ticker}")
    markets = response.json().get('markets', [])
    
    all_minute_data = []
    
    print(f"\nFound {len(markets)} markets")
    print("Fetching 1-minute candlesticks for each market...\n")
    
    for i, market in enumerate(markets, 1):
        ticker = market['ticker']
        subtitle = market.get('subtitle', market.get('title', 'N/A'))
        
        # Get market timing
        open_time = datetime.fromisoformat(market['open_time'].replace('Z', '+00:00'))
        close_time = datetime.fromisoformat(market['close_time'].replace('Z', '+00:00'))
        start_ts = int(open_time.timestamp())
        end_ts = int(close_time.timestamp())
        
        print(f"[{i}/{len(markets)}] {subtitle}")
        
        # Fetch 1-minute candlesticks
        candle_url = f"{base_url}/series/{series_ticker}/markets/{ticker}/candlesticks"
        candle_response = requests.get(
            candle_url,
            params={
                "start_ts": start_ts,
                "end_ts": end_ts,
                "period_interval": 1  # 1-minute candles
            }
        )
        
        if candle_response.status_code == 200:
            candlestick_json = candle_response.json()
            candles = candlestick_json.get('candlesticks', [])
            
            if candles:
                print(f"  ✓ Got {len(candles)} sparse 1-minute candles")
                
                # Convert to DataFrame
                for candle in candles:
                    candle_time = datetime.fromtimestamp(
                        candle['end_period_ts'], 
                        tz=timezone.utc
                    )
                    
                    price_obj = candle.get('price', {})
                    yes_bid_obj = candle.get('yes_bid', {})
                    yes_ask_obj = candle.get('yes_ask', {})
                    
                    all_minute_data.append({
                        'ticker': ticker,
                        'subtitle': subtitle,
                        'timestamp': candle_time,
                        'price_close': price_obj.get('close'),
                        'yes_bid_close': yes_bid_obj.get('close'),
                        'yes_ask_close': yes_ask_obj.get('close'),
                        'volume': candle.get('volume', 0),
                        'open_interest': candle.get('open_interest', 0)
                    })
            else:
                print(f"  ⚠️  No 1-minute data")
        else:
            print(f"  ✗ HTTP {candle_response.status_code}")
        
        time.sleep(0.2)
    
    if all_minute_data:
        # Create DataFrame from sparse data
        df_sparse = pd.DataFrame(all_minute_data)
        
        print(f"\n{'='*80}")
        print("FORWARD FILLING TO CREATE CONTINUOUS DATA")
        print("="*80)
        
        # Process each market separately
        all_filled_data = []
        
        for ticker in df_sparse['ticker'].unique():
            market_data = df_sparse[df_sparse['ticker'] == ticker].copy()
            subtitle = market_data['subtitle'].iloc[0]
            
            # Create complete minute range
            min_time = market_data['timestamp'].min()
            max_time = market_data['timestamp'].max()
            
            # Generate all minutes in range
            full_range = pd.date_range(start=min_time, end=max_time, freq='1min')
            
            # Create filled DataFrame
            df_filled = pd.DataFrame({'timestamp': full_range})
            
            # Merge with actual data
            df_filled = df_filled.merge(
                market_data[['timestamp', 'price_close', 'yes_bid_close', 'yes_ask_close', 'volume', 'open_interest']], 
                on='timestamp', 
                how='left'
            )
            
            # Forward fill prices
            df_filled['price_close'] = df_filled['price_close'].ffill()
            df_filled['yes_bid_close'] = df_filled['yes_bid_close'].ffill()
            df_filled['yes_ask_close'] = df_filled['yes_ask_close'].ffill()
            
            # Calculate median price (mid of bid/ask)
            df_filled['median_price'] = (df_filled['yes_bid_close'] + df_filled['yes_ask_close']) / 2
            
            # Fill volume/OI with 0 for missing minutes
            df_filled['volume'] = df_filled['volume'].fillna(0)
            df_filled['open_interest'] = df_filled['open_interest'].ffill()
            
            # Add metadata
            df_filled['ticker'] = ticker
            df_filled['subtitle'] = subtitle
            
            all_filled_data.append(df_filled)
            
            sparse_count = len(market_data)
            filled_count = len(df_filled)
            print(f"\n{subtitle}:")
            print(f"  Sparse data: {sparse_count} candles")
            print(f"  Filled data: {filled_count} candles")
            print(f"  Coverage: {sparse_count/filled_count*100:.1f}% had actual data")
        
        # Combine all markets
        df_nyc_1min = pd.concat(all_filled_data, ignore_index=True)
        
        # Sort by ticker and timestamp
        df_nyc_1min = df_nyc_1min.sort_values(['ticker', 'timestamp']).reset_index(drop=True)
        
        print(f"\n{'='*80}")
        print(f"✓ FINAL DATASET: {len(df_nyc_1min)} continuous 1-minute candles")
        print("="*80)
        
        print(f"\nDataFrame shape: {df_nyc_1min.shape}")
        print(f"Columns: {list(df_nyc_1min.columns)}")
        print(f"\nMarkets: {df_nyc_1min['subtitle'].unique().tolist()}")
        print(f"Date range: {df_nyc_1min['timestamp'].min()} to {df_nyc_1min['timestamp'].max()}")
        
        print(f"\n{'='*80}")
        print("SAMPLE DATA (First 10 rows)")
        print("="*80)
        print(df_nyc_1min[['timestamp', 'subtitle', 'median_price', 'yes_bid_close', 'yes_ask_close', 'volume']].head(10).to_string())
        
        print(f"\n💾 Data stored in: df_nyc_1min")
        print("="*80)
        
    else:
        print("\n❌ No 1-minute data available")
        df_nyc_1min = pd.DataFrame()

except Exception as e:
    print(f"\n❌ Error: {e}")
    import traceback
    traceback.print_exc()
    df_nyc_1min = pd.DataFrame()

FETCHING 1-MINUTE CANDLESTICKS WITH FORWARD FILL

Found 5 markets
Fetching 1-minute candlesticks for each market...

[1/5] :: Democratic Nominee
  ✗ HTTP 400
[2/5] :: Republican Nominee
  ✗ HTTP 400
[3/5] :: Independent
  ✗ HTTP 400
[4/5] :: Independent
  ✗ HTTP 400
[5/5] :: Independent
  ✗ HTTP 400

❌ No 1-minute data available


In [19]:
# ============================================================================
# FETCH ALL CANDLESTICK DATA - NYC MAYOR PARTY WINNER
# ============================================================================
def fetch_market_candlestick_data(event_ticker):
    print("="*80)
    print("FETCHING ALL CANDLESTICK DATA - NYC MAYOR PARTY WINNER")
    print("="*80)

    import pandas as pd

    event_ticker = "KXMAYORNYCPARTY-25"
    base_url = "https://api.elections.kalshi.com/trade-api/v2"

    # Extract series ticker (remove year suffix)
    series_ticker = event_ticker.rsplit('-', 1)[0]  # "KXMAYORNYCPARTY"

    all_candlestick_data = []

    try:
        # Fetch event and markets
        response = requests.get(f"{base_url}/events/{event_ticker}")
        
        if response.status_code == 200:
            event_data = response.json()
            markets = event_data.get('markets', [])
            
            print(f"\nSeries: {series_ticker}")
            print(f"Found {len(markets)} markets in event")
            print("Fetching candlestick data for each market...\n")
            
            for i, market in enumerate(markets, 1):
                ticker = market['ticker']
                subtitle = market.get('subtitle', market.get('title', 'N/A'))
                
                # Get market timing
                open_time = datetime.datetime.fromisoformat(market['open_time'].replace('Z', '+00:00'))
                close_time = datetime.datetime.fromisoformat(market['close_time'].replace('Z', '+00:00'))
                start_ts = int(open_time.timestamp())
                end_ts = int(close_time.timestamp())
                
                print(f"[{i}/{len(markets)}] {subtitle} ({ticker})")
                
                # Try to fetch candlestick data with different periods
                for period in [60, 1440]:  # 1-hour and daily
                    try:
                        # Correct endpoint: /series/{series_ticker}/markets/{ticker}/candlesticks
                        candle_url = f"{base_url}/series/{series_ticker}/markets/{ticker}/candlesticks"
                        candle_response = requests.get(
                            candle_url,
                            params={
                                "start_ts": start_ts,
                                "end_ts": end_ts,
                                "period_interval": period
                            }
                        )
                        
                        if candle_response.status_code == 200:
                            candlestick_json = candle_response.json()
                            candles = candlestick_json.get('candlesticks', [])
                            
                            if candles:
                                print(f"  ✓ Got {len(candles)} candlesticks (period={period} min)")
                                
                                # Add to collection
                                for candle in candles:
                                    candle_time = datetime.datetime.fromtimestamp(
                                        candle['end_period_ts'], 
                                        tz=datetime.timezone.utc
                                    )
                                    
                                    # Extract price data
                                    price_obj = candle.get('price', {})
                                    yes_bid_obj = candle.get('yes_bid', {})
                                    yes_ask_obj = candle.get('yes_ask', {})
                                    
                                    all_candlestick_data.append({
                                        'ticker': ticker,
                                        'series_ticker': series_ticker,
                                        'party': subtitle,
                                        'timestamp': candle_time,
                                        'timestamp_unix': candle['end_period_ts'],
                                        'period_minutes': period,
                                        # Trade prices (actual executed trades)
                                        'price_open': price_obj.get('open'),
                                        'price_high': price_obj.get('high'),
                                        'price_low': price_obj.get('low'),
                                        'price_close': price_obj.get('close'),
                                        # YES bid prices (buyer's side)
                                        'yes_bid_open': yes_bid_obj.get('open'),
                                        'yes_bid_high': yes_bid_obj.get('high'),
                                        'yes_bid_low': yes_bid_obj.get('low'),
                                        'yes_bid_close': yes_bid_obj.get('close'),
                                        # YES ask prices (seller's side)
                                        'yes_ask_open': yes_ask_obj.get('open'),
                                        'yes_ask_high': yes_ask_obj.get('high'),
                                        'yes_ask_low': yes_ask_obj.get('low'),
                                        'yes_ask_close': yes_ask_obj.get('close'),
                                        'volume': candle.get('volume', 0),
                                        'open_interest': candle.get('open_interest', 0)
                                    })
                                break
                            else:
                                print(f"  ⚠️  No data for period={period}")
                        else:
                            print(f"  ⚠️  HTTP {candle_response.status_code} for period={period}")
                            if candle_response.status_code != 404:
                                print(f"      Response: {candle_response.text[:200]}")
                            
                    except Exception as e:
                        print(f"  ✗ Error fetching period={period}: {e}")
                        continue
                
                time.sleep(0.2)  # Rate limiting
            
            # Create DataFrame
            if all_candlestick_data:
                df_candlesticks = pd.DataFrame(all_candlestick_data)
                
                print(f"\n{'='*80}")
                print(f"✓ FETCHED {len(df_candlesticks)} TOTAL CANDLESTICKS")
                print("="*80)
                
                print(f"\nDataFrame shape: {df_candlesticks.shape}")
                print(f"Columns: {list(df_candlesticks.columns)}")
                print(f"\nParties: {df_candlesticks['party'].unique().tolist()}")
                print(f"Date range: {df_candlesticks['timestamp'].min()} to {df_candlesticks['timestamp'].max()}")
                
                print(f"\n{'='*80}")
                print("SAMPLE DATA (First 10 rows)")
                print("="*80)
                print(df_candlesticks.head(10).to_string())
                
                print(f"\n💾 Data stored in: df_candlesticks")
            else:
                print("\n❌ No candlestick data retrieved")
                df_candlesticks = pd.DataFrame()
                
        else:
            print(f"\n❌ Error fetching event: HTTP {response.status_code}")
            df_candlesticks = pd.DataFrame()

    except Exception as e:
        print(f"\n❌ Error: {e}")
        import traceback
        traceback.print_exc()
        df_candlesticks = pd.DataFrame()
    return df_candlesticks

    print("\n" + "="*80)

In [64]:
# ============================================================================
# STREAM LIVE BTCUSD PRICE UPDATES
# ============================================================================
# Uses the /api/v1/streaming endpoint with HTTP chunked encoding
# Press Ctrl+C (or stop the cell) to stop streaming
# ============================================================================

import json
from datetime import datetime

PRECISION = 1e18

def stream_prices(symbols: list = ["BTCUSD"], duration_seconds: int = None):
    """
    Stream live price updates from Chainlink Candlestick API.
    
    Parameters:
    -----------
    symbols : list
        List of symbols to stream (e.g., ["BTCUSD", "ETHUSD"])
    duration_seconds : int
        How long to stream (None = indefinitely until interrupted)
    """
    if candlestick_api is None or candlestick_api.token is None:
        print("❌ API not initialized. Run the authorization cell first.")
        return
    
    # Ensure we have a valid token
    candlestick_api._ensure_authorized()
    
    symbols_str = ",".join(symbols)
    url = f"{candlestick_api.base_url}/api/v1/streaming?symbol={symbols_str}"
    
    headers = {
        "Authorization": f"Bearer {candlestick_api.token}",
        "Connection": "keep-alive"
    }
    
    print("="*70)
    print(f"STREAMING LIVE PRICES: {symbols_str}")
    print("="*70)
    print(f"URL: {url}")
    print(f"Press Ctrl+C or stop the cell to end streaming\n")
    
    start_time = time.time()
    update_count = 0
    
    try:
        # Use stream=True for chunked encoding
        with candlestick_api.session.get(url, headers=headers, stream=True, timeout=None) as response:
            if response.status_code != 200:
                print(f"❌ Error: {response.status_code} - {response.text}")
                return
            
            print("✅ Connected! Waiting for price updates...\n")
            
            # Read streaming response line by line
            for line in response.iter_lines(decode_unicode=True):
                if line:
                    try:
                        data = json.loads(line)
                        
                        # Check if it's a heartbeat
                        if "heartbeat" in data:
                            hb_time = datetime.fromtimestamp(data["heartbeat"])
                            print(f"💓 Heartbeat: {hb_time.strftime('%H:%M:%S')}", end="\r")
                        
                        # Check if it's a trade/price update
                        elif data.get("f") == "t":
                            symbol = data.get("i", "???")
                            raw_price = data.get("p", 0)
                            timestamp = data.get("t", 0)
                            
                            # Convert to USD
                            price_usd = raw_price / PRECISION
                            dt = datetime.fromtimestamp(timestamp)
                            
                            update_count += 1
                            print(f"🔵 {symbol}: ${price_usd:,.2f}  |  {dt.strftime('%Y-%m-%d %H:%M:%S')}  |  Update #{update_count}")
                        
                        else:
                            # Unknown message type
                            print(f"📨 {data}")
                    
                    except json.JSONDecodeError:
                        print(f"⚠️ Could not parse: {line[:100]}")
                
                # Check duration limit
                if duration_seconds and (time.time() - start_time) >= duration_seconds:
                    print(f"\n⏱️ Duration limit ({duration_seconds}s) reached. Stopping.")
                    break
    
    except KeyboardInterrupt:
        print(f"\n\n🛑 Streaming stopped by user")
    except Exception as e:
        print(f"\n❌ Error: {e}")
    finally:
        elapsed = time.time() - start_time
        print(f"\n{'='*70}")
        print(f"Streamed for {elapsed:.1f} seconds, received {update_count} price updates")
        print("="*70)


# Stream BTCUSD prices for 60 seconds (change to None for indefinite)
stream_prices(["BTCUSD"], duration_seconds=60)

STREAMING LIVE PRICES: BTCUSD
URL: https://priceapi.dataengine.chain.link/api/v1/streaming?symbol=BTCUSD
Press Ctrl+C or stop the cell to end streaming

✅ Connected! Waiting for price updates...

🔵 BTCUSD: $78,558.48  |  2026-02-02 12:45:10  |  Update #1
🔵 BTCUSD: $78,559.95  |  2026-02-02 12:45:12  |  Update #2
🔵 BTCUSD: $78,560.19  |  2026-02-02 12:45:14  |  Update #3
🔵 BTCUSD: $78,564.82  |  2026-02-02 12:45:16  |  Update #4
🔵 BTCUSD: $78,563.69  |  2026-02-02 12:45:18  |  Update #5
🔵 BTCUSD: $78,563.71  |  2026-02-02 12:45:20  |  Update #6
🔵 BTCUSD: $78,565.69  |  2026-02-02 12:45:22  |  Update #7
🔵 BTCUSD: $78,569.52  |  2026-02-02 12:45:24  |  Update #8
🔵 BTCUSD: $78,570.25  |  2026-02-02 12:45:26  |  Update #9
🔵 BTCUSD: $78,573.23  |  2026-02-02 12:45:28  |  Update #10
🔵 BTCUSD: $78,574.46  |  2026-02-02 12:45:30  |  Update #11
🔵 BTCUSD: $78,576.20  |  2026-02-02 12:45:32  |  Update #12


🛑 Streaming stopped by user

Streamed for 26.2 seconds, received 12 price updates
